# Projection onto the simplex



<div class="alert alert-block alert-success">
This notebook implements an algorithm for the projection of a vector $y$ onto the simplex $\Delta$ with
\begin{align}
    \Delta := \Big\{ (x_1,\ldots,x_N)\in \mathbb{R}^N:\, \sum_{n=1}^N x_n =a \quad\text{and}\quad x\geq 0 \Big\}.
\end{align}
    
The implemented algorithm was proposed in: Condat, L. <i>Fast projection onto the simplex and the ℓ<sub>1</sub> ball</i>. Math. Program. 158, 575–585 (2016). [https://doi.org/10.1007/s10107-015-0946-6](https://doi.org/10.1007/s10107-015-0946-6). 
    
The paper is also available at [https://hal.archives-ouvertes.fr/hal-01056171v2](https://hal.archives-ouvertes.fr/hal-01056171v2).
</div>

<div style="background-color:rgba(0, 0, 0, 0.0470588); vertical-align: middle; padding:5px 0; padding-left: 40px;">
<h2 style="color: #5e9ca0;">Condat's Algorithm</h2>
<ol>
<li>Set $v:= (y_1)$, $u$ as an empty list, $\rho:= y_1 - a$.</li>
<li>For $n \in \{2,\ldots, N\}$, do
<ol>
<li>If $y_n > \rho$
<ol>
<li>Set $\rho := \rho + (y_n - \rho)/(|v|+1)$.</li>
<li>If $\rho > y_n - a$, add $y_n$ to $v$.</li>
<li>Else, add $v$ to $u$, set $v = (y_n)$, $\rho = y_n -a$.</li>
</ol>
</li>
</ol>
</li>
<li>If $u$ is not empty, for every element $y$ of $u$, do
<ol>
<li>If $y > \rho$, add $y$ to $v$ and set $\rho := \rho + (y-\rho)/|v|$.</li>
</ol>
</li>
<li>Do, while $|v|$ changes,
<ol>
<li>For every element $y$ of $v$ do
<ol>
<li>If $y\leq \rho$, remove $y$ from $v$ and set $\rho := \rho + (\rho - y)/|v|$.</li>
</ol>
</li>
</ol>
</li>
<li>Set $\tau := \rho$, $K = |v|$.</li>
<li>For $n \in \{1,\ldots,N\}$, set $x_n := \max \{y_n - \tau, 0\}$.</li>
</ol>
</div>

The algorithm will be tested projecting a random vector $y$ of size $N$ where $y_n \sim \mathcal{U}(-1,2)$ for all $n\in \{1,\ldots,N\}$.

In [1]:
import numpy as np
from numpy import nditer, ndenumerate, zeros, zeros_like, clip, allclose, full, maximum
from numpy.random import default_rng as rng

In [2]:
N = 1000
y = rng(0).uniform(-1,2,N)
a = 1.0

## List version

<div class="alert alert-block alert-info">
First implementation is list-based. It sticks to the algorithm above, and thus it cannot take the advantage that $v$ is the list of positive values of $y$, thus requiring the unnecessary triming in Step 6.
</div>

In [3]:
# Step 1
v = [y[0]]
u = []
ρ = y[0] - a

In [4]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( len(v) + 1 )
        if ρ > yₙ - a:
            v.append(yₙ)
        else:
            u.extend(v)
            v = [yₙ]
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for ι in iter(u):
        if ι > ρ:
            v.append(ι)
            ρ += (ι - ρ)/( len(v) )
# Step 4
while True:
    ℓ_v = len(v)
    for ı in range(ℓ_v - 1, -1, -1):
        if v[ı] <= ρ:
            ρ += (ρ - v[ı])/(len(v) - 1)
            del v[ı]
    if len(v) >= ℓ_v:
        break

In [5]:
x = clip(y - ρ, 0, None)

In [6]:
v - ρ

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

## Vectorial version 1

<div class="alert alert-block alert-info">
The following adaptation introduces some array operations to avoid the extra step of clipping negative numbers. Here $v$ is not a set but a boolean array of the same size as $y$. Its cells determine if a cell of $y$ is active in $v$ or not. In the end, $x$ is computed just on the active cells given by $v$.
</div>

In [7]:
# Step 1
ρ = y[0] - a

v = zeros(N, dtype=bool)
x = zeros_like(y)
u = []
v[0] = True

In [8]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( v.sum() + 1 )
        if ρ > yₙ - a:
            v[n] = True
        else:
            u.extend([i[0] for i,j in ndenumerate(v) if j])
            v *= False;    v[n] = True
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for n in iter(u):
        yₙ = y[n]
        if yₙ > ρ:
            v[n] = True
            ρ += (yₙ - ρ)/( v.sum() )
# Step 4
while True:
    ℓ_v = v.sum()
    for i,j in ndenumerate(v):
        if j:
            if y[i[0]] <= ρ:
                ρ += (ρ - y[i[0]])/(v.sum() - 1)
                v[i[0]] = False
    if v.sum() >= ℓ_v:
        break

In [9]:
x[v] = y[v] - ρ #clip(y[v] - ρ, 0.0, None)

In [10]:
x[v]

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

## Vectorial version 2

<div class="alert alert-block alert-info">
The final implementation takes back the original idea of using lists instead of an array of the same size as $y$. In this case, $v$ does not contain values from $y$ but its indeces. This way a sparse representation of $v$ is achieved and $x$ does not go through a clipping step as well.
</div>

In [11]:
# Step 1
ρ = y[0] - a

x = zeros_like(y)
v = [0]
u = []

In [12]:
# Step 2
for n in range(1,N):
    yₙ = y[n]
    if yₙ > ρ:
        ρ += (yₙ - ρ)/( len(v) + 1 )
        if ρ > yₙ - a:
            v.append(n)
        else:
            u.extend(v)
            v = [n]
            ρ = yₙ - a
# Step 3
if len(u) > 0:
    for n in iter(u):
        yₙ = y[n]
        if yₙ > ρ:
            v.append(n)
            ρ += (yₙ - ρ)/( len(v) )
# Step 4
while True:
    ℓ_v = len(v)
    for i,j in enumerate(v):
        if y[j] <= ρ:
            ρ += (ρ - y[j])/(len(v) - 1)
            del v[i]
    if len(v) >= ℓ_v:
        break

In [13]:
x[v] = y[v] - ρ #clip(y[v] - ρ, 0.0, None)

In [14]:
x[v]

array([0.06744738, 0.01832359, 0.06110709, 0.01940269, 0.00520454,
       0.06056962, 0.02164565, 0.01061472, 0.04190357, 0.05789789,
       0.02130254, 0.0148837 , 0.06684845, 0.05188726, 0.00241651,
       0.04805173, 0.0173415 , 0.02582169, 0.03312648, 0.07432163,
       0.02410941, 0.01056192, 0.03883018, 0.03691761, 0.05087173,
       0.00455268, 0.06171698, 0.05232123])

---

## Performance tests

<div class="alert alert-block alert-warning">
Now we check which routine is the fastest.
</div>

In [15]:
def CondatP_a(y,a,N):
    # Step 1
    v = [y[0]]
    u = []
    ρ = y[0] - a
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(yₙ)
            else:
                u.extend(v)
                v = [yₙ]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for ι in iter(u):
            if ι > ρ:
                v.append(ι)
                ρ += (ι - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for ı in range(ℓ_v - 1, -1, -1):
            if v[ı] <= ρ:
                ρ += (ρ - v[ı])/(len(v) - 1)
                del v[ı]
        if len(v) >= ℓ_v:
            break
            
    return clip(y - ρ, 0, None)

def CondatP_b(y,a,N):
    # Step 1
    ρ = y[0] - a

    v = zeros(N, dtype=bool)
    x = zeros_like(y)
    u = []
    v[0] = True
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( v.sum() + 1 )
            if ρ > yₙ - a:
                v[n] = True
            else:
                u.extend([i[0] for i,j in ndenumerate(v) if j])
                v *= False;    v[n] = True
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v[n] = True
                ρ += (yₙ - ρ)/( v.sum() )
    # Step 4
    while True:
        ℓ_v = v.sum()
        for i,j in ndenumerate(v):
            if j:
                if y[i[0]] <= ρ:
                    ρ += (ρ - y[i[0]])/(v.sum() - 1)
                    v[i[0]] = False
        if v.sum() >= ℓ_v:
            break

    x[v] = y[v] - ρ 
    return x

def CondatP_c(y,a,N):
    # Step 1
    ρ = y[0] - a

    x = zeros_like(y)
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    x[v] = y[v] - ρ
    return x

In [16]:
%timeit -r 10 -n 200 CondatP_a(y,a,N)

389 µs ± 152 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [17]:
%timeit -r 10 -n 200 CondatP_b(y,a,N)

1.57 ms ± 91.9 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [18]:
%timeit -r 10 -n 200 CondatP_c(y,a,N)

387 µs ± 42.1 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


As we can see, the last implementation might be a little faster than the original.

---

## Just-in-time compilation with numba

<div class="alert alert-block alert-warning">
We use numba to speed up the computation time.
</div>

In [19]:
from numba import jit, prange

In [20]:
@jit(nopython=True, fastmath = True, parallel = True)
def CondatP_a(y,a,N):
    # Step 1
    v = [y[0]]
    u = []
    ρ = y[0] - a
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(yₙ)
            else:
                u.extend(v)
                v = [yₙ]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for ι in iter(u):
            if ι > ρ:
                v.append(ι)
                ρ += (ι - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for ı in range(ℓ_v - 1, -1, -1):
            if v[ı] <= ρ:
                ρ += (ρ - v[ı])/(len(v) - 1)
                del v[ı]
        if len(v) >= ℓ_v:
            break
            
    return maximum(y - ρ, 0)

In [21]:
@jit(nopython=True, fastmath = True, parallel = True)
def CondatP_b(y,a,N):
    # Step 1
    ρ = y[0] - a

    v = full(N, False)
    x = zeros_like(y)
    u = []
    v[0] = True
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( v.sum() + 1 )
            if ρ > yₙ - a:
                v[n] = True
            else:
                u.extend([i[0] for i,j in ndenumerate(v) if j])
                v *= False;    v[n] = True
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v[n] = True
                ρ += (yₙ - ρ)/( v.sum() )
    # Step 4
    while True:
        ℓ_v = v.sum()
        for i,j in ndenumerate(v):
            if j:
                if y[i[0]] <= ρ:
                    ρ += (ρ - y[i[0]])/(v.sum() - 1)
                    v[i[0]] = False
        if v.sum() >= ℓ_v:
            break

    x[v] = y[v] - ρ 
    return x

In [22]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def CondatP_c(y,a,N):
    # Step 1
    ρ = y[0] - a

    x = zeros_like(y)
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    #x[v] = y[v] - ρ
    for n in iter(v):
        x[n] = y[n] - ρ
    #for n in prange(len(v)):
    #    x[v[n]] = y[v[n]] - ρ
    return x

In [23]:
# Run things once for pre-compiling:
CondatP_a(y,a,N), CondatP_b(y,a,N), CondatP_c(y,a,N);

Now we see a clear winner.

In [24]:
%timeit -r 10 -n 200 CondatP_a(y,a,N)

110 µs ± 20.7 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [25]:
%timeit -r 10 -n 200 CondatP_b(y,a,N)

9.71 ms ± 320 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [26]:
%timeit -r 10 -n 200 CondatP_c(y,a,N)

29.9 µs ± 6.18 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


Finally, a sanity check:

In [27]:
allclose(CondatP_a(y,a,N), CondatP_b(y,a,N)), allclose(CondatP_a(y,a,N), CondatP_c(y,a,N))

(True, True)

---

Now we can finally addapt the code to accept a matrix $\gamma$ as input.

In [80]:
N = 1000
M = 500
γ = rng(0).uniform(-1,2,(N,M))
a = 1.0

In [81]:
y = rng(0).uniform(-1,2,N)
x = zeros_like(γ)

In [64]:
@jit(nopython=True, fastmath = True)#, parallel = True)
def CondatP_c(y,x,a,N):
    # Step 1
    ρ = y[0] - a
    v = [0]
    u = []
    # Step 2
    for n in range(1,N):
        yₙ = y[n]
        if yₙ > ρ:
            ρ += (yₙ - ρ)/( len(v) + 1 )
            if ρ > yₙ - a:
                v.append(n)
            else:
                u.extend(v)
                v = [n]
                ρ = yₙ - a
    # Step 3
    if len(u) > 0:
        for n in iter(u):
            yₙ = y[n]
            if yₙ > ρ:
                v.append(n)
                ρ += (yₙ - ρ)/( len(v) )
    # Step 4
    while True:
        ℓ_v = len(v)
        for i,j in enumerate(v):
            if y[j] <= ρ:
                ρ += (ρ - y[j])/(len(v) - 1)
                del v[i]
        if len(v) >= ℓ_v:
            break
    
    #x[v] = y[v] - ρ
    for n in iter(v):
        x[n] = y[n] - ρ
    #for n in prange(len(v)):
    #    x[v[n]] = y[v[n]] - ρ
    return x

In [65]:
CondatP_c(y,x,a,N); # Null test

In [82]:
for i in range(N):
    x[i] = CondatP_c(γ[i],x[i],a,M)

In [83]:
# Just checking :)
for i in range(N):
    if x[i].sum() > 2:
        print(i)

In [84]:
x.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

---